In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import BaggingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd

In [2]:
users_behavior = pd.read_csv('users_behavior.csv')

In [3]:
users_behavior.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
X = users_behavior[['calls', 'minutes', 'messages', 'mb_used']]
y = users_behavior['is_ultra']

In [6]:
from sklearn.model_selection import train_test_split

# разобьем данные на обучающую и тестовую выборку
# размер тестовой выборки составит 30%
# также зададим точку отсчета для воспроизводимости
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [7]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [8]:
print(lr.score(X_valid, y_valid))

0.7316062176165803


После Логистической регресии метрика качества accuracy = 0.73 Запомним это как результат для сравнения с обучениями для этого датасета.

**Stacking (стекинг)**

In [41]:
estimators = [('lr', LogisticRegression()), ('dt', DecisionTreeClassifier())]
modelClf = StackingClassifier(estimators=estimators, final_estimator=SVC())

In [42]:
modelClf.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('dt', DecisionTreeClassifier())],
                   final_estimator=SVC())

In [43]:
print(modelClf.score(X_valid, y_valid))

0.7295336787564767


Результат без подбора параметров 0.7295 чуть хуже чем у обычной логистичесекой регресии. Проведем подбор параметров для улучшения качества предсказаний

In [12]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
'''
estimators = [('lr', LogisticRegression(solver = "newton-cg", penalty = "l2", C = 0.01)), ('dt', DecisionTreeClassifier(criterion = 'entropy', max_depth = 5))]
modelClf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())

params = {'lr__solver': ['liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga', 'lbfgs'],
          'lr__penalty':['l2'],
          'lr__C': [0.01, 0.1, 1, 10, 100], 
          'dt__criterion': ['entropy', 'gini'],
          'dt__max_depth': [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

grid = GridSearchCV(estimator=modelClf, param_grid=params, cv=5, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

print("parameter ", grid_result.best_params_)
'''

'\nestimators = [(\'lr\', LogisticRegression(solver = "newton-cg", penalty = "l2", C = 0.01)), (\'dt\', DecisionTreeClassifier(criterion = \'entropy\', max_depth = 5))]\nmodelClf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())\n\nparams = {\'lr__solver\': [\'liblinear\', \'newton-cg\', \'newton-cholesky\', \'sag\', \'saga\', \'lbfgs\'],\n          \'lr__penalty\':[\'l2\'],\n          \'lr__C\': [0.01, 0.1, 1, 10, 100], \n          \'dt__criterion\': [\'entropy\', \'gini\'],\n          \'dt__max_depth\': [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}\n\ngrid = GridSearchCV(estimator=modelClf, param_grid=params, cv=5, scoring=\'accuracy\')\ngrid_result = grid.fit(X_train, y_train)\n\nprint("parameter ", grid_result.best_params_)\n'

После подбора параметров лучшими из 
params = {'lr__solver': ['liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga', 'lbfgs'],
          'lr__penalty':['l2'],
          'lr__C': [0.01, 0.1, 1, 10, 100], 
          'dt__criterion': ['entropy', 'gini'],
          'dt__max_depth': [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
оказались 
parameter  {'dt__criterion': 'entropy', 'dt__max_depth': 5, 'lr__C': 0.1, 'lr__penalty': 'l2', 'lr__solver': 'sag'}

In [50]:
#parameter  {'dt__criterion': 'entropy', 'dt__max_depth': 5, 'lr__C': 0.1, 'lr__penalty': 'l2', 'lr__solver': 'sag'}
estimators = [('lr', LogisticRegression(C = 0.1, penalty = "l2", solver = "sag")), ('dt', DecisionTreeClassifier(criterion = 'entropy', max_depth = 5))]
modelClf = StackingClassifier(estimators=estimators, final_estimator=SVC())

In [51]:
modelClf.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was re

StackingClassifier(estimators=[('lr', LogisticRegression(C=0.1, solver='sag')),
                               ('dt',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=5))],
                   final_estimator=SVC())

In [52]:
print(modelClf.score(X_valid, y_valid))

0.7792746113989637


Результат Stacking с подобранными параметрами оказался  0.7792, что в лучшую сторону отличается от 0.7295.

**Bagging (бэггинг)**

In [16]:
modelClf = BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=50, random_state=12)

In [17]:
modelClf.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=50,
                  random_state=12)

In [18]:
print(modelClf.score(X_valid, y_valid))

0.7139896373056995


Результат без подбора параметров 0.7139 чуть хуже чем у обычной логистичесекой регресии.

**Random Forest (случайный лес)**


In [19]:
clf = RandomForestClassifier(n_estimators=20, max_depth=6,
                             min_samples_split=2, random_state=0)

In [20]:
scores = cross_val_score(clf, X_train, y_train, cv=3)

In [21]:
scores.mean()

0.8096899569796766

In [22]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=20, random_state=0)

In [23]:
print(clf.score(X_valid, y_valid))

0.783419689119171


Подбор гиперпараметров для случайного леса (через цикл и кросс-валидацию)

In [24]:
best_score = 0
best_params = None

for max_depth in range(2, 10):
    for n_estimators in range(10, 31, 10):
        model_rfr = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, random_state=12345)
        score = cross_val_score(model_rfr, X_train, y_train, cv=3, n_jobs=-1).mean()
        if score > best_score:
            best_score = score
            best_params = {'max_depth': max_depth, 'n_estimators': n_estimators}

print('Лучшее значение accuracy для случайного леса: {} при значениях гиперпараметров: {}'.format(best_score, best_params))

Лучшее значение accuracy для случайного леса: 0.8110262572318647 при значениях гиперпараметров: {'max_depth': 6, 'n_estimators': 10}


Результат с подбором параметров 0.8110. Результат стал значительно лучше чем у просто Логистической регресии

**Boosting (бустинг)**

**Adaboost (адаптивный бустинг)**

In [25]:
modelClf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=12)

In [26]:
modelClf.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   n_estimators=100, random_state=12)

In [27]:
print(modelClf.score(X_valid, y_valid))

0.7626943005181347


Результат без подбора параметров 0.7626 немного лучше чем у просто Логистической регресии

**Gradient Boosting (Градиентный бустинг)**

In [28]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1,
                                 max_depth=1, random_state=0)

In [29]:
clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1, max_depth=1, random_state=0)

In [30]:
print(clf.score(X_valid, y_valid))

0.7813471502590673


Результат без подбора параметров 0.7813. Результат лучше чем у просто Логистической регрессии

**Voting (простое усреднение)**

In [31]:
decisiontree = DecisionTreeClassifier(max_depth=2)

In [32]:
forest = RandomForestClassifier(n_estimators=20, max_depth=6,
                             min_samples_split=2, random_state=0)

In [33]:
ensemble=VotingClassifier(estimators=[('Decision Tree', decisiontree), ('Random Forest', forest)], 
                       voting='soft', weights=[1,1]).fit(X_train, y_train)

In [34]:
print(ensemble.score(X_valid, y_valid))

0.7689119170984456


Результат без подбора параметров 0.7689. Результат лучше чем у просто Логистической регрессии

Попробовав разные способы, оказалось что их результат близок или достаточно лучше чем у обычной логистической регресии.
Так же удалось улучшить результат Stacking до 0.7792 после подбора параметров, что значительно лучше чем результат 0.7295 до подбора параметров